In [1]:
import os
from io import StringIO # if going with no saving csv file
import dask.dataframe as dd
from dask.delayed import delayed
from pathlib import Path
import pandas as pd
import os
from functools import reduce

In [2]:
from pprint import pprint, pp
from IPython.display import display, Markdown
from icecream import ic

In [3]:
%%capture 

from nanoHUB.application import Application
from nanoHUB.dataaccess.sql import SqlDataFrameMapper, QueryParams
from nanoHUB.dataaccess.sql import CachedConnection, TunneledConnection
application = Application.get_instance()

In [4]:
%%capture 

rfm_engine = application.new_db_engine('rfm_data')
nanohub_engine = application.new_db_engine('nanohub')
metrics_engine = application.new_db_engine('nanohub_metrics')

cache_dir = Path(os.getenv('APP_DIR') + '/.cache')

In [ ]:
toolstart_df = pd.read_sql_query(
    "SELECT id, datetime, user, tool, walltime, cputime FROM nanohub_metrics.toolstart",
    nanohub_engine
)
toolstart_df['datetime'] = pd.to_datetime(toolstart_df['datetime'],errors='coerce')
toolstart_df.to_parquet(PurePath(cache_dir, 'toolstart_1'), compression='gzip')

In [5]:
tool_version_df = pd.read_sql_query(
    "SELECT  toolname, instance FROM nanohub.jos_tool_version",
    nanohub_engine
)

In [6]:
connection = CachedConnection(TunneledConnection(
    ssh_host = os.getenv('tunnel_ssh_host'),
    ssh_username = os.getenv('tunnel_ssh_username'),
    ssh_password = os.getenv('tunnel_ssh_password'),
    ssh_port = int(os.getenv('tunnel_ssh_port')),
    remote_bind_address = os.getenv('tunnel_remote_bind_address'),
    remote_bind_port = int(os.getenv('tunnel_remote_bind_port')),
    db_host = os.getenv('db_host'),
    db_username = os.getenv('db_user'),
    db_password = os.getenv('db_password')
))

def data_mapper(chunksize: int = 1000):
    return SqlDataFrameMapper(connection, chunksize)

default_data_mapper = data_mapper()

In [7]:
params_toolstart = QueryParams(
    db_name='nanohub_metrics', table_name='toolstart', col_names = ['id', 'datetime', 'user', 'tool', 'walltime', 'cputime'], index_key='id'
)

tool_version_df['instance'].str.lower()
tool_version_df['toolname'].str.lower()


# for partial_df in default_data_mapper.map(params_toolstart):
#     display(partial_df)
    

,toolname,instance
0,,
1,112016a,112016a_dev
2,1dbtetransient,1dbtetransient_dev
3,1dchainmd,1dchainmd_dev
4,1dchainmd,1dchainmd_r13
...,...,...
5130,zooleyjupyter,zooleyjupyter_dev
5131,zooleylinux,zooleylinux_dev
5132,zooleylinux1,zooleylinux1_dev
5133,zooleylinux1,zooleylinux1_r4


,toolname,instance
0,,
1,112016a,112016a_dev
2,1dbtetransient,1dbtetransient_dev
3,1dchainmd,1dchainmd_dev
4,1dchainmd,1dchainmd_r13
...,...,...
5130,zooleyjupyter,zooleyjupyter_dev
5131,zooleylinux,zooleylinux_dev
5132,zooleylinux1,zooleylinux1_dev
5133,zooleylinux1,zooleylinux1_r4
